## 서울시 기상 데이터 전처리

### 데이터 전처리 (Data Preprocessing)

이 노트북은 서울시 기상 데이터를 분석에 적합한 형태로 전처리하기 위해 작성되었습니다.

- 데이터 로드 및 기본 정보 확인
- 날짜 형식 변환 및 정렬

전처리를 통해 모델링에 필요한 깨끗하고 일관된 형태의 데이터셋을 구축하는 데 목적이 있습니다.

### 1. 데이터 로드 및 기본 설정

In [1]:
#  Jupyter Notebook에서 외부 모듈(.py 파일)을 수정할 경우,
#  매번 커널을 재시작하거나 수동으로 reload하지 않아도
#  자동으로 가장 최신 상태로 import 되도록 설정합니다.

# %load_ext autoreload:
# IPython의 autoreload 확장 기능을 불러옵니다.
# 이 기능은 외부 .py 파일이 변경될 때 자동으로 다시 import 하도록 도와줍니다.

# %autoreload 2:
# 모든 모듈을 cell 실행 시마다 자동으로 reload합니다.
# (즉, 외부 .py 파일을 수정하고 저장만 해도 바로 반영됨)

# 사용 예:
#   - utils.py, visualization.py 등 자주 수정하는 모듈이 있는 경우
#   - 매번 커널 재시작 없이 편리하게 함수 변경 사항을 반영하고 싶을 때

# 단, 모듈의 내부 상태를 유지하고 싶은 경우에는 주의해서 사용해야 합니다.

%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# 프로젝트 루트 경로를 sys.path에 추가
project_root = os.path.abspath("..")  # notebooks 폴더 기준 상위 폴더
if project_root not in sys.path:
    sys.path.append(project_root)

In [3]:
import pandas as pd
from scripts.utils import *

# 한글 폰트 설정
setup_font()

Current OS: Darwin
Setting macOS font: AppleGothic
Current font settings: ['AppleGothic']


In [4]:
# 원본 데이터가 저장된 폴더 경로 설정
folder_path = '../data/raw/weather'
save_dir = '../data/processed/weather'

# 처리할 서울시 구별 기상 데이터 파일
# 서울시와 종로구의 기상 데이터가 개별 CSV 파일로 저장되어 있음
# 서울시 통합 데이터에 결측치 (종로구 측정소인 북한산 2018.12.04 이후로 측정 중단) 존재
# 종관기상관측(ASOS)의 서울 측정소(종로에 위치) 데이터 사용하여 결측치 보완하였기 때문에 종로구를 제외한 서울시, 종로구 데이터 별도 존재
seoul_file = 'Seoul_daily_weather_2018_2024.csv'
jongno_file = 'Jongno_daily_weather_2018_2024.csv'

# 파일 읽기
seoul_df = pd.read_csv(os.path.join(folder_path, seoul_file), encoding='cp949')
jongno_df = pd.read_csv(os.path.join(folder_path, jongno_file), encoding='cp949')

# 저장 폴더 생성
os.makedirs(save_dir, exist_ok=True)

### 2. 서울시(종로구 제외 전체 구)와 종로구 대기질 데이터 기본 전처리 및 통합 후 저장
- 날짜 형식 변환
- 필요한 컬럼 선택
- 기본적인 데이터 정제
- 종로구, 서울시 데이터 통합
- 통합 데이터 저장

In [5]:
# 데이터프레임의 열 이름에서 공백 제거
strip_column_names(seoul_df)
strip_column_names(jongno_df)

# 서울시 데이터 처리
# 서울시 지점명 정제
seoul_df['지점명'] = seoul_df['지점명'].str.replace(r'강북\*', '강북', regex=True)
seoul_df['지점명'] = seoul_df['지점명'].str.replace(r'현충원', '동작', regex=True)
seoul_df['지점명'] = seoul_df['지점명'].str.replace(r'남현', '관악', regex=True)

# 종로구 데이터 처리
# 종로구 지점명 정제
jongno_df['지점명'] = jongno_df['지점명'].str.replace(r'서울', '종로', regex=True)

# 서울시 데이터에 종로구 데이터 추가
df = pd.concat([seoul_df, jongno_df], ignore_index=True)

# 서울시 데이터 컬럼명 변경
df = df.rename(columns={
    '일시': '날짜',
})
df = df[['지점명', '날짜', '평균기온(°C)', '일강수량(mm)', '평균 풍속(m/s)']]

df = to_datetime_column(df, date_col= '날짜')

# 전처리된 데이터 저장
# '../data/processed/weather' 디렉토리에 저장
save_to_csv(
    df,
    output_dir='../data/processed/weather',
   file_name='daily_weather_preprocessed'
)
